<a href="https://colab.research.google.com/github/FenNlay/STT/blob/train_custom_data/Kopie_van_train_your_first_coqui_STT_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train your first 🐸 STT model 💫

👋 Hello and welcome to Coqui (🐸) STT 

The goal of this notebook is to show you a **typical workflow** for **training** and **testing** an STT model with 🐸.

Let's train a very small model on a very small amount of data so we can iterate quickly.

In this notebook, we will:

1. Download data and format it for 🐸 STT.
2. Configure the training and testing runs.
3. Train a new model.
4. Test the model and display its performance.

So, let's jump right in!

*PS - If you just want a working, off-the-shelf model, check out the [🐸 Model Zoo](https://www.coqui.ai/models)*

In [ ]:
## Install Coqui STT
! pip install -U pip
! pip install coqui_stt_training

## ✅ Download & format sample data for English

**First things first**: we need some data.

We're training a Speech-to-Text model, so we need some _speech_ and we need some _text_. Specificially, we want _transcribed speech_. Let's download an English audio file and its transcript and then format them for 🐸 STT. 

🐸 STT expects to find information about your data in a CSV file, where each line contains:

1. the **path** to an audio file
2. the **size** of that audio file
3. the **transcript** of that audio file.

Formatting the audio and transcript isn't too difficult in this case. We define `download_sample_data()` which does all the work. If you have a custom dataset, you will want to write a custom data importer.

**Second things second**: we want an alphabet. The output layer of a typical* 🐸 STT model represents letters in the alphabet. Let's download an English alphabet from Coqui and use that.

*_If you are working with languages with large character sets (e.g. Chinese), you can set `bytes_output_mode=True` instead of supplying an `alphabet.txt` file. In this case, the output layer of the STT model will correspond to individual UTF-8 bytes instead of individual characters._

In [ ]:
### Download sample data
import os
import pandas
import urllib.request
from coqui_stt_training.util.downloader import maybe_download

### Import data
!curl -o cv_en.txt "https://datasets-server.huggingface.co/first-rows?dataset=common_voice&config=en&split=validated"

def download_sample_data():
    data_dir="english/"

    # Download data + alphabet
    fhandle = open("cv_en.txt")
    for line in fhandle:
      x = line.split("],")
      y = line.split(",{")
      
    sentences = []
    for i in x:
      if i.startswith("\"sentence"):
        i = i.replace("\"sentence\":\"", "")
        i = i.replace("\\\"", "")
        stp = "\""
        i = i.split(stp, 1)[0]
        sentences.append(i)
          
    audios = []
    count = 0
    for j in y:
      if j.startswith("\"src"):
        j = j.replace("\"src\":\"", "")
        stp = "\""
        j = j.split(stp, 1)[0]
        audios.append(j)
    
    a = 0
    audio_file = []
    for l in audios:
      name = f"audio{a}.wav"
      file = maybe_download(f"{name}", data_dir, f"{l}")
      audio_file.append(file)
      a += 1

    b = 0
    transcript_file = []
    for k in sentences:
      name = f"audio{b}.txt"
      with open(f"/content/{data_dir}{name}", "w") as wf:
        wf.write(f"{k}")
      transcript_file.append(f"{data_dir}{name}")
      b += 1


    alphabet = maybe_download("alphabet.txt", data_dir, "https://raw.githubusercontent.com/coqui-ai/STT/main/data/alphabet.txt")
    
    # Format data
    df = pandas.DataFrame(data=[(1, 1)],
                          columns=["wav_filename", "wav_filesize"])
    for m in audio_file:
      new_row = {"wav_filename": os.path.abspath(m), "wav_filesize": os.path.getsize(m)}
      df = df.append(new_row, ignore_index=True)
    trans = [1]
    for n in transcript_file:
      with open(n, "r") as fin:
        transcript = " ".join(fin.read().strip().lower().split(" ")).replace(".", "").replace(",", "").replace("’", "").replace("?", "").replace("!", "").replace("-", "")
        trans.append(transcript)
    df['transcript'] = trans
    df = df.iloc[1: , :]
        
    # Save formatted CSV 
    df.to_csv(os.path.join(data_dir, "data_cv.csv"), index=False)

# Download and format data
download_sample_data()

### 👀 Take a look at the data

In [ ]:
csv_file = open("english/data_cv.csv", "r")
print(csv_file.read())

In [ ]:
alphabet_file = open("english/alphabet.txt", "r")
print(alphabet_file.read())

## ✅ Configure & set hyperparameters

Coqui STT comes with a long list of hyperparameters you can tweak. We've set default values, but you will often want to set your own. You can use `initialize_globals_from_args()` to do this. 

You must **always** configure the paths to your data, and you must **always** configure your alphabet. Additionally, here we show how you can specify the size of hidden layers (`n_hidden`), the number of epochs to train for (`epochs`), and to initialize a new model from scratch (`load_train="init"`).

In [90]:
from coqui_stt_training.util.config import initialize_globals_from_args

initialize_globals_from_args(
    alphabet_config_path="english/alphabet.txt",
    checkpoint_dir="ckpt_dir",
    train_files=["english/data_cv.csv"],
    dev_files=["english/data_cv.csv"],
    test_files=["english/data_cv.csv"],
    load_train="init",
    n_hidden=200,
    epochs=100,
)

### 👀 View all Config settings

In [ ]:
from coqui_stt_training.util.config import Config

# Take a peek at the entire Config
print(Config.to_json())

## ✅ Train a new model

Let's kick off a training run 🚀🚀🚀 (using the configure you set above).

This notebook should work on either a GPU or a CPU. However, in case you're running this on _multiple_ GPUs we want to only use one, because the sample dataset (one audio file) is too small to split across multiple GPUs.

In [ ]:
from coqui_stt_training.train import train

# use maximum one GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

train()

## ✅ Test the model

We made it! 🙌

Let's kick off the testing run, which displays performance metrics.

We're committing the cardinal sin of ML 😈 (aka - testing on our training data) so you don't want to deploy this model into production. In this notebook we're focusing on the workflow itself, so it's forgivable 😇

You can see from the test output that our tiny model has overfit to the data, and basically memorized this one sentence.

When you start training your own models, make sure your testing data doesn't include your training data 😅

In [ ]:
from coqui_stt_training.evaluate import test

test()